# EDA

In [ ]:
import os
import pandas as pd
len(os.listdir('../input/severstal-steel-defect-detection/train_images'))

In [ ]:
import pandas as pd
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
df

In [ ]:
df['ClassId'].value_counts()

In [ ]:
# Example Photo
import matplotlib.pyplot as plt
import cv2

pathh = '000a4bcdd.jpg'

img = cv2.imread('../input/severstal-steel-defect-detection/train_images/'+pathh)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.imshow(img)


pathh = '008ef3d74.jpg'
img = cv2.imread('../input/severstal-steel-defect-detection/train_images/'+pathh)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.imshow(img)

pathh = '0014fce06.jpg'
img = cv2.imread('../input/severstal-steel-defect-detection/train_images/'+pathh)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.imshow(img)


pathh = '000f6bf48.jpg'
img = cv2.imread('../input/severstal-steel-defect-detection/train_images/'+pathh)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.imshow(img)


In [ ]:
img

# Model

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!pip install botnoi==0.2.1

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import botnoi as bn
from botnoi import cv
import os

In [ ]:
import pandas as pd
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
df

In [ ]:
df0 = df[df['ClassId'] == 1].reset_index(drop=True)
df1 = df[df['ClassId'] == 2].reset_index(drop=True)
df2 = df[df['ClassId'] == 3].reset_index(drop=True)
df3 = df[df['ClassId'] == 4].reset_index(drop=True)

In [ ]:
df2 = df2.sample(frac=1)[:1000].reset_index(drop=True)

In [ ]:
df = pd.concat([df0, df1, df2, df3], ignore_index=True)
df

In [ ]:
df['ClassId'].value_counts()

In [ ]:
import glob
# imgfolder = glob.glob('../input/severstal-steel-defect-detection/train_images/'+'*')
# imgfolder

In [ ]:
name = df['ImageId'].tolist()
classes = df['ClassId'].tolist()
# name

In [ ]:
import glob
import pandas as pd
import pickle
from tqdm import tqdm

def createdataset(df):
    
    pList = df['ImageId'].tolist()
    classes = df['ClassId'].tolist()

    clsset = pd.DataFrame()
    featvec = []
    clss = []
    for p in tqdm(range(0, len(pList))):
        clss.append(classes[p])
        a = cv.image('../input/severstal-steel-defect-detection/train_images/'+pList[p])
        featvec.append(a.getresnet50())
        

    clsset['feature'] = featvec
    clsset['clss'] = clss
    #return clsset
#     dataset.append(clsset)
    #return dataset
#     return pd.concat(dataset,axis=0)
    return clsset


In [ ]:
feavec = createdataset(df)

In [ ]:
dataset = feavec
dataset = dataset.rename(columns={'clss': 'label'})
# dataset['label'].astype('str').dtypes
# dataset.info()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
def trainmodel(dataset,modfile=''):
    trainfeat, testfeat, trainlabel, testlabel = train_test_split(dataset['feature'], dataset['label'], test_size=0.33,random_state=42) #random_state=42
    return trainfeat,testfeat,trainlabel,testlabel
#     clf = LinearSVC()
#     clf = CalibratedClassifierCV(clf) 
#     mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)
#     #return mod
# #     res = mod.predict(np.vstack(testfeat.values))
    
#     #return res, testlabel
# #     acc = sum(res == testlabel)/len(res)
# #     conf = mod.predict_proba(np.vstack(testfeat.values))
#     return mod#,acc,conf,testlabel

In [ ]:
trainfeat, testfeat, trainlabel, testlabel = trainmodel(dataset)

In [ ]:
clf = LinearSVC()
clf = CalibratedClassifierCV(clf)
mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)

In [ ]:
 res = mod.predict(np.vstack(testfeat.values))

In [ ]:
acc = sum(res == testlabel)/len(res)
acc

In [ ]:
sample = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
sample

In [ ]:
import os
len(os.listdir('../input/severstal-steel-defect-detection/test_images'))

In [ ]:
real = sample[['ImageId']]
# real

In [ ]:
def createdataset(df):
    
    pList = df['ImageId'].tolist()
#     classes = df['ClassId'].tolist()

    clsset = pd.DataFrame()
    featvec = []
#     clss = []
    for p in tqdm(range(0, len(pList))):
#         clss.append(classes[p])
        a = cv.image('../input/severstal-steel-defect-detection/test_images/'+pList[p])
        featvec.append(a.getresnet50())
        

    clsset['feature'] = featvec
#     clsset['clss'] = clss
    #return clsset
#     dataset.append(clsset)
    #return dataset
#     return pd.concat(dataset,axis=0)
    return clsset

In [ ]:
fea = createdataset(real)

In [ ]:
fea = fea['feature']
fea

In [ ]:
ans = mod.predict(np.vstack(fea.values))

In [ ]:
real['ans'] = ans
real

In [ ]:
real.to_csv('ans_metal.csv', index=False)